In [1]:
!pip install -r requirements.txt

  Using cached spicy-0.16.0-py2.py3-none-any.whl (1.7 kB)


ERROR: Could not find a version that satisfies the requirement json (from -r requirements.txt (line 6)) (from versions: none)
ERROR: No matching distribution found for json (from -r requirements.txt (line 6))


In [1]:
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from tokenizers import ByteLevelBPETokenizer
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt

In [3]:
from dataset import encoding
from model_joint import make_model
from model_joint import LabelSmoothing
from model_joint import NoamOpt
from model_joint import data_gen
from belief_accuracy import joint_accuracy
from belief_accuracy import slot_accuracy
import torch

**Dataloader cell**

In [6]:
s1,t1,a1= encoding('simpletod/Processed_data/simptod/context_train.txt','simpletod/Processed_data/simptod/belief_train.txt','simpletod/Processed_data/simptod/action_train.txt',400)    #Train data
s9,t9,a9 = encoding('simpletod/Processed_data/simptod/context_val.txt','simpletod/Processed_data/simptod/belief_val.txt','simpletod/Processed_data/simptod/action_val.txt',400)        #Validation data
s4,t4,a4 = encoding('simpletod/Processed_data/simptod/context_test.txt','simpletod/Processed_data/simptod/belief_test.txt','simpletod/Processed_data/simptod/action_test.txt',400)      #Test data

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [10]:
#can be ignored
tokenizer = ByteLevelBPETokenizer('vocab.json','merges.txt')
tokenizer.add_special_tokens(["<PAD>","<|belief|>"," <|endofbelief|>","<|context|>"," <|user|> "," <|system|> ","<|endofcontext|>","<|action|>"," <|endofaction|>"])

9

In [11]:
pad_tok = tokenizer.encode("<PAD>").ids.pop()
print(pad_tok)

18157


In [12]:
def run_epoch(data_iter, model, loss_compute):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    loss1 = []
    index = []
    ref1 = []
    cand1 = []
    #file1 = open("test_res6.txt", "a")
    for i, batch in enumerate(data_iter):
        reference = []
        candidate = []
        reference1 = []
        candidate1 = []
        ref2 = []
        cand2 = []
        out_belief, out_action = model.forward_belief_action(batch.src.to(device), batch.trg1.to(device), 
                            batch.src_mask.to(device), batch.trg1_mask.to(device),batch.trg2.to(device),batch.trg2_mask.to(device))
        #out_action = model.forward_action(batch.src.to(device), batch.trg2.to(device), 
                            #batch.src_mask.to(device), batch.trg2_mask.to(device))
        loss = loss_compute(out_belief,out_action, batch.trg1_y.to(device),batch.trg2_y.to(device), batch.ntokens1.to(device),batch.ntokens2.to(device))
        loss1.append(loss)
        index.append(i)
        total_loss += loss
        total_tokens += (batch.ntokens1+batch.ntokens2)
        tokens += (batch.ntokens1+batch.ntokens2)
        #file1.write("Batch No. - %d \n" % (i+1))
        #max , idx = torch.max(model.generator1(out_action),dim=2)
        #pred2 = idx.tolist()
        #for d in range(len(batch.src.tolist())):
                #file1.write(''.join(list(print_ele(batch.src.tolist()[d]))) + "\n")#tokenizer.decode(batch.src.tolist()[d],skip_special_tokens=False) + "\n")
                #file1.write("<|belief|>" + tokenizer.decode(pred2[d],skip_special_tokens=True) + " <|endofbelief|>" + "\n")
                #file1.write("<|belief|>" + tokenizer.decode(batch.trg_y.tolist()[d],skip_special_tokens=True) + " <|endofbelief|>" + "\n")
                #file1.write("<|action|>" + tokenizer.decode(pred2[d],skip_special_tokens=True) + " <|endofaction|>" + "\n")
                #file1.write("<|action|>" + tokenizer.decode(batch.trg2_y.tolist()[d],skip_special_tokens=True) + " <|endofaction|>" + "\n")
                #ref1.append(tokenizer.decode(pred2[d],skip_special_tokens=True).split(','))
                #cand1.append(tokenizer.decode(batch.trg_y.tolist()[d],skip_special_tokens=True).split(','))
                #ref2.append(tokenizer.decode(pred2[d],skip_special_tokens=True).split(','))
                #cand2.append(tokenizer.decode(batch.trg_y.tolist()[d],skip_special_tokens=True).split(','))
        #file1.write("Joint Accuracy - %f \n" % joint_accuracy(cand2,ref2,d_c = False,type2_c = False))
        #file1.write("Slot Accuracy - %f \n" % slot_accuracy(cand2,ref2))
        if i % 200 == 1:
            elapsed = time.time() - start
            """"max1 , idx1 = torch.max(model.generator(out_belief),dim=2)
            max2 , idx2 = torch.max(model.generator1(out_action),dim=2)

            a1 = idx1.tolist()
            b1 = batch.trg1_y.tolist()
            a2 = idx2.tolist()
            b2 = batch.trg2_y.tolist()

            #print(idx.size())
            #print(batch.trg_y.size())
            for j in range(len(a1)):
                reference.append(tokenizer.decode(a1[j],skip_special_tokens=True).split(','))
                candidate.append(tokenizer.decode(b1[j],skip_special_tokens=True).split(','))
            for p in range(len(a2)):
                reference1.append(tokenizer.decode(a2[p],skip_special_tokens=True).split(','))
                candidate1.append(tokenizer.decode(b2[p],skip_special_tokens=True).split(',')) 
            print(reference)
            print(candidate)
            print(reference1)
            print(candidate1)""""
            accuracy_belief = joint_accuracy(candidate,reference,d_c = False,type2_c = False)
            slt_accuracy_belief = slot_accuracy(candidate,reference)
            #accuracy_action = joint_accuracy(candidate1,reference1,d_c = False,type2_c = False)
            #slt_accuracy_action = slot_accuracy(candidate1,reference1)

            print("Epoch Step: %d Loss: %f Tokens per Sec: %f Joint_belief: %f Slot_belief: %f" %
                    (i, loss / (batch.ntokens1+batch.ntokens2), tokens / elapsed,accuracy_belief, slt_accuracy_belief))
            start = time.time()
            tokens = 0
    #file1.write("Overall Joint Accuracy - %f \n" % joint_accuracy(cand1,ref1,d_c = False,type2_c = False))
    #file1.write("Overall Slot Accuracy - %f \n" % slot_accuracy(cand1,ref1))
    #file1.close()
    return total_loss / total_tokens

In [14]:
file9 = open("loss_file8.txt", "w") 
L = ["This is a file containing losses and accuracies of train and validation data \n"] 
file9.writelines(L)
file9.close()

In [15]:
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator,generator1, criterion, opt=None):
        self.generator = generator
        self.generator1 = generator1
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x1,x2, y,z, norm1,norm2):
        x1 = self.generator(x1)
        x2 = self.generator1(x2)
        loss1 = self.criterion(x1.contiguous().view(-1, x1.size(-1)), 
                              y.contiguous().view(-1)) / norm1
        loss2 = self.criterion(x2.contiguous().view(-1, x2.size(-1)), 
                              z.contiguous().view(-1)) / norm2
        loss = loss1 + loss2
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.item() * norm1

In [6]:
model = torch.load('simpletod/dialog_NLP.pt')     #run this cell if there is a previously saved model like the previous cell

In [16]:
V = 30000
model = make_model(V, V, N=2) #run this cell if there is no saved model like the previous cell

C:\Users\alipu\Desktop\TransformerDialog_ArnabM\JointBeliefAct\model_joint.py:315: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)


In [ ]:
criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)

model_opt = NoamOpt(model.src_embed[0].d_model, 1, 400,
        torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9))
#q = 7
model.to(device)
for epoch in range(50):
    model.train()
    run_epoch(data_gen(V, 10, 5670,s1,t1,a1), model, 
              SimpleLossCompute(model.generator,model.generator1, criterion, model_opt))
    model.eval()
    l1 = run_epoch(data_gen(V, 10,737 ,s1,t1,a1), model, 
              SimpleLossCompute(model.generator,model.generator1, criterion, model_opt))
    l = run_epoch(data_gen(V, 10, 737,s9,t9,a9), model, 
                    SimpleLossCompute(model.generator,model.generator1, criterion, None))
    torch.save(model,'simpletod/dialog_NLP.pt')
    file1 = open("loss_now.txt", "a")
    file1.write("Epoch %d \n" % (epoch+1)) # append mode 
    file1.write("Train loss %f \n" % l1) 
    file1.write("Validation loss %f \n" % l)